In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from datetime import datetime
from datetime import timezone
from datetime import timedelta
#monitoring
import mflib
from mflib.mflib import MFLib

fablib = fablib_manager()
#fablib.show_config();

In [10]:
#Create Slice
slice = fablib.new_slice(name="slice_TCP")

# Create 2 nodes
node1 = slice.add_node(name="node1", site="UCSD", cores=8, ram=16)
node2 = slice.add_node(name="node2", site="CERN", cores=8, ram=16)


#iface1 = node1.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]
iface1 = node1.add_component(model='NIC_ConnectX_6', name="nic1").get_interfaces()[0]
iface2 = node2.add_component(model='NIC_ConnectX_6', name="nic2").get_interfaces()[0]


slice.add_l3network(name="net1", interfaces=[iface1], type='IPv6')
slice.add_l3network(name="net2", interfaces=[iface2], type='IPv6')

# Add Measurement node
MFLib.addMeasNode(slice, site="UCSD", disk=100)

#Submit Slice Request
slice.submit();


Retry: 1, Time: 43 sec


ID,57d3ffc9-ffc2-443f-aecd-1ddb47ef1022
Name,slice_TCP2
Lease Expiration (UTC),2024-09-17 18:20:09 +0000
Lease Start (UTC),2024-09-16 18:20:09 +0000
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
State,Closing


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
0fdb42bd-45ed-497b-87f2-911b3fd05a0f,meas-node,4,16,100,docker_ubuntu_20,qcow2,ucsd-w2.fabric-testbed.net,UCSD,ubuntu,,Closed,TicketReviewPolicy: Closing reservation due to failure in slice#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
16e56908-43a3-4c61-8514-923eceb59ad4,node1,None,None,None,default_rocky_8,qcow2,None,UCSD,rocky,,Closed,Insufficient resources : Component of type: ConnectX-6 not available in graph node: J64W1N3#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
306fdd4f-aaec-407c-8750-1c6ab0c4f6ae,node2,None,None,None,default_rocky_8,qcow2,None,CERN,rocky,,Closed,Insufficient resources : Component of type: ConnectX-6 not available in graph node: 3B5F6R3#,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
815177d8-8b18-435f-8d05-3fa6b2a73e37,l3_meas_net_CERN,L3,FABNetv4,CERN,l3_meas_net_CERN.subnet,l3_meas_net_CERN.gateway,Closed,"redeem predecessor reservation# 306fdd4f-aaec-407c-8750-1c6ab0c4f6ae is in a terminal state, failing the reservation# 815177d8-8b18-435f-8d05-3fa6b2a73e37#"
22a05de7-5717-45a7-ba6e-a324a173a763,l3_meas_net_UCSD,L3,FABNetv4,UCSD,l3_meas_net_UCSD.subnet,l3_meas_net_UCSD.gateway,Closed,"redeem predecessor reservation# 16e56908-43a3-4c61-8514-923eceb59ad4 is in a terminal state, failing the reservation# 22a05de7-5717-45a7-ba6e-a324a173a763#"
7823bc7f-8696-4580-85b4-1b67b5e38f03,net1,L3,FABNetv6,UCSD,net1.subnet,net1.gateway,Closed,"redeem predecessor reservation# 16e56908-43a3-4c61-8514-923eceb59ad4 is in a terminal state, failing the reservation# 7823bc7f-8696-4580-85b4-1b67b5e38f03#"
0886a3d7-f1af-4507-ae85-6a0f8108de66,net2,L3,FABNetv6,CERN,net2.subnet,net2.gateway,Closed,"redeem predecessor reservation# 306fdd4f-aaec-407c-8750-1c6ab0c4f6ae is in a terminal state, failing the reservation# 0886a3d7-f1af-4507-ae85-6a0f8108de66#"


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
node1-nic1-p1,p1,node1,None,100,config,,,,,,None,None
node1-nic1-p2,p2,node1,net1,100,config,,,,,,None,p2
node1-meas_nic_node1_UCSD-p1,p1,node1,l3_meas_net_UCSD,100,auto,,,,,,None,p1
node2-nic2-p1,p1,node2,net2,100,config,,,,,,None,p1
node2-nic2-p2,p2,node2,None,100,config,,,,,,None,None
node2-meas_nic_node2_CERN-p1,p1,node2,l3_meas_net_CERN,100,auto,,,,,,None,p1
meas-node-meas_nic_meas-node_UCSD-p1,p1,meas-node,l3_meas_net_UCSD,100,auto,,,,,,6,p1



Time to print interfaces 43 seconds


In [2]:
slice = fablib.get_slice(name="slice_TCP")


In [3]:
slice.show()

ID,8e9e1651-d6aa-4198-becb-90528ad448ff
Name,slice_TCP
Lease Expiration (UTC),2024-09-23 18:32:46 +0000
Lease Start (UTC),2024-09-09 17:51:26 +0000
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
State,StableOK


ID,8e9e1651-d6aa-4198-becb-90528ad448ff
Name,slice_TCP
Lease Expiration (UTC),2024-09-23 18:32:46 +0000
Lease Start (UTC),2024-09-09 17:51:26 +0000
Project ID,a57c7715-d871-4369-82e6-408c9a57a6e7
State,StableOK


In [ ]:
# Get the objects again (not sure why)
node1 = slice.get_nodes()[0]
node2 = slice.get_nodes()[1]

net1 = slice.get_network(name="net1")
net2 = slice.get_network(name="net2")

# Get 1 available IP for each interface
avail_ip1 = net1.get_available_ips()[0]
avail_ip2 = net2.get_available_ips()[0]

In [ ]:
mf = MFLib("slice_TCP", mf_repo_branch="main")
print(mf.grafana_tunnel)
instrumetize_results = mf.instrumentize()
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

In [8]:
print(avail_ip1)
print(avail_ip2)

2602:fcfb:d:1::2
2602:fcfb:1d:1::2


In [ ]:
# Network setup node1
commands = []
commands.append(f'sudo ip -6 addr add {avail_ip1}/64 dev eth1')
commands.append(f'sudo bash -c "echo 1\tsense1 >> /etc/iproute2/rt_tables"')
commands.append(f'sudo ip -6 route add default via {str(avail_ip1)[:-1]+"1"} dev eth1 table sense1')
commands.append(f'sudo ip -6 rule add to {str(avail_ip1)+"/128"} table sense1')
commands.append(f'sudo ip -6 rule add from {str(avail_ip1)+"/128"} table sense1')
commands.append(f'sudo ip link set mtu 9000 eth1')
commands.append(f'sudo ip -6 rule add to {str(avail_ip2)+"/128"} table sense1')
commands.append(f'sudo ip -6 rule add from {str(avail_ip2)+"/128"} table sense1')


for command in commands:
    print(command)
    stdout, stderr = node1.execute(command)

In [ ]:
# Network setup node2
commands = []
commands.append(f'sudo ip -6 addr add {avail_ip2}/64 dev eth1')
commands.append(f'sudo bash -c "echo 1\tsense1 >> /etc/iproute2/rt_tables"')
commands.append(f'sudo ip -6 route add default via {str(avail_ip2)[:-1]+"1"} dev eth1 table sense1')
commands.append(f'sudo ip -6 rule add to {str(avail_ip2)+"/128"} table sense1')
commands.append(f'sudo ip -6 rule add from {str(avail_ip2)+"/128"} table sense1')
commands.append(f'sudo ip link set mtu 9000 eth1')
commands.append(f'sudo ip -6 rule add to {str(avail_ip1)+"/128"} table sense1')
commands.append(f'sudo ip -6 rule add from {str(avail_ip1)+"/128"} table sense1')
for command in commands:
    print(command)
    stdout, stderr = node2.execute(command)

In [7]:

stdout, stderr = node2.execute(f'ping -I {avail_ip2} -c 5 {avail_ip1}')

AuthenticationException: Authentication failed.

In [ ]:
# non-dockerize XRootD
commands = []
##commands.append(f'sudo yum install https://repo.opensciencegrid.org/osg/23-main/osg-23-main-el8-release-latest.rpm -y')
##commands.append(f'sudo yum install https://dl.fedoraproject.org/pub/epel/epel-release-latest-8.noarch.rpm -y')
commands.append(f'sudo yum install wget iputils iproute java -y')
commands.append(f'cd / && sudo wget -4 http://monalisa.cern.ch/FDT/lib/fdt.jar')
for command in commands:
    print(command)
    stdout, stderr = node2.execute(command)
    stdout, stderr = node1.execute(command)

In [4]:
# Print the SSH command to log into the node
print({node1.get_ssh_command()})

{'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@132.249.252.142'}


In [5]:
# Print the SSH command to log into the node
print({node2.get_ssh_command()})

{'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe00:2f15'}


In [ ]:
# Set end host to now plus 15 days
end_date = (datetime.now(timezone.utc) + timedelta(days=15)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [6]:
# Show lease end date/time
print(f"Lease End (UTC)        : {slice.get_lease_end()}")

Lease End (UTC)        : 2024-09-23 18:32:46 +0000


In [ ]:
slice.show()